In [17]:
from ansys.mapdl.core import launch_mapdl
import os

script_dir = r'C:\Pranav2.0\8'
mapdl = launch_mapdl(run_location=script_dir, override=True)


In [18]:
# 1. Enter AUX15 for IGES import
mapdl.aux15()
mapdl.igesin('Design1.igs')

# 2. Return to BEGIN (the default processor)
mapdl.finish()  # This brings you back to the BEGIN level

# 3. Now enter PREP7 for element and meshing commands
mapdl.prep7()


*****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [19]:
# Define shell element properties
mapdl.et(1, "SHELL181")
mapdl.sectype(1, "SHELL")
mapdl.secdata(0.000125)  # Set thickness

# Set element size to 1 mm
mapdl.esize(5.0)  # Uniform element size

# Select all areas (faces) and mesh them
mapdl.asel("ALL")  # Select all areas
mapdl.amesh("ALL")  # Mesh all selected areas

print("Meshing complete. All faces meshed with 1mm elements.")

Meshing complete. All faces meshed with 1mm elements.


In [20]:
# 6. Plot the mesh
mesh_plot = mapdl.eplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x13784bc6ae0_5&reconnect=auto" class="pyvis…

In [21]:
# 3. Export the mesh to CDB in the script's directory
cdb_filename = "Design1_mesh"
mapdl.cdwrite("db", cdb_filename, "cdb")

# 4. Confirm the file location
exported_file = os.path.join(script_dir, cdb_filename + ".cdb")
if os.path.exists(exported_file):
    print(f"Exported CDB file: {exported_file}")
else:
    print("Error: CDB file not found in script directory!")

Exported CDB file: C:\Pranav2.0\8\Design1_mesh.cdb


In [22]:
print(mapdl.geometry)

MAPDL Selected Geometry
Keypoints:  45
Lines:      42
Areas:      9
Volumes:    0



In [23]:
mapdl.pnum('area', 1)
mapdl.alist()

LIST ALL SELECTED AREAS.
   *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION
 NUMBER LOOP  LINES                     AREA       ELEM SIZE    #NODES  #ELEM    MAT   REAL   TYP  ESYS   SECN
     10  1     31     30     29     28   N/A         0.000        1035   1898     -1     -1    -1     0     -1
     11  1     43     42     41     40   N/A         0.000        1031   1855     -1     -1    -1     0     -1
     12  1     29     54     43     52   N/A         0.000         624   1365     -1     -1    -1     0     -1
     13  1     41     66     31     64   N/A         0.000         249    574     -1     -1    -1     0     -1
     14  1     79     78     77     76  0.5523E+05   0.000        1944   2144     -1     -1    -1     0     -1
         2     87     86     85     84
     15  1     96     42     98     87   N/A         0.000         244    560     -1     -1    -1     0     -1
     16  1     54    109     86     98   N/A         0.000         323    745  

In [24]:
#see from the above alist and select the area of the frame fillet using hit and trial
mapdl.asel('S', 'AREA', '', 15,18)
mapdl.nsla('s', 1)
mapdl.nplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x13784bc7fb0_6&reconnect=auto" class="pyvis…

In [25]:
mapdl.n('100000',0,0,0)
mapdl.n('100001',1e-2,0,0)
mapdl.n('100002',1e-2,1e-2,0)
mapdl.cs('8001',1,100000,100001,100002)
mapdl.csys(8001)

mapdl.ndele(100000,100002,1)
    

DELETE SELECTED NODES FROM     100000 TO     100002 BY          1

In [26]:
mapdl.cmdele('all')
def frame_sel():
    mapdl.asel('S', 'AREA', '', 15,18)
    mapdl.nsla('s', 1)

frame_sel()
mapdl.nsel('r','loc','y',-0.5,0.5)
mapdl.cm('te_1','node')
mapdl.cmplot('te_1','node')


[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x13784bde6c0_7&reconnect=auto" class="pyvis…

In [27]:
frame_sel()
mapdl.nsel('r','loc','y',175,185)
mapdl.cm('le_1','node')
mapdl.cmplot('le_1','node')

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x13784bde5d0_8&reconnect=auto" class="pyvis…

In [28]:
for i in range (2,16):
    frame_sel()
    mapdl.nsel('r','loc','y',(i/2)-0.5,i/2) # 1 degree 
    mapdl.cm(f'te_r_{i}','node')
    frame_sel()
    mapdl.nsel('r','loc','y',-((i/2)-0.5),-i/2) # 1 degree
    mapdl.cm(f'te_l_{i}','node')
    frame_sel()
    mapdl.nsel('r','loc','y',180+(2.5*i),180+(5*i)) # 5 degree
    mapdl.cm(f'le_l_{i}','node')
    frame_sel()
    mapdl.nsel('r','loc','y',180-(2.5*i),180-(5*i)) # 5 degree
    mapdl.cm(f'le_r_{i}','node')

In [29]:
mapdl.cmplot('all','node')
components = mapdl.cmlist('all',1,'node')
with open('mapdl_components.txt', 'w') as f:
    f.write(components)

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x13784bddb20_9&reconnect=auto" class="pyvis…

In [30]:
import re
import csv

# Replace this with reading from a file if needed
component_text = components
components = {}
current_comp = None
for line in component_text.split('\n'):
    line = line.strip()
    if not line:
        continue
    # Detect component name line
    comp_match = re.match(r'^([A-Z0-9_]+)\s+YES\s+NODE$', line)
    if comp_match:
        current_comp = comp_match.group(1)
        components[current_comp] = []
        continue
    # Parse node numbers lines
    if current_comp:
        nodes = re.findall(r'\d+', line)
        components[current_comp].extend(nodes)

# Write to CSV
with open('components_nodes.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Component', 'Node'])
    for comp, nodes in components.items():
        for node in nodes:
            writer.writerow([comp, node])

print("CSV file 'components_nodes.csv' created successfully.")


CSV file 'components_nodes.csv' created successfully.


In [31]:
mapdl.exit()


In [32]:
#this code is just because there is some update issue, so to avoid error, etblock has to be changed

def replace_etblock_in_cdb(input_cdb_path: str, output_cdb_path: str):
    """
    Reads an ANSYS CDB file, finds the 'ETBLOCK' section, and replaces the
    entire block with a single 'ET,1,SHELL181' command.

    This is used to fix compatibility issues with older parsers that do not
    recognize the ETBLOCK format.

    Args:
        input_cdb_path (str): The full path to the original .cdb file.
        output_cdb_path (str): The full path where the modified .cdb file will be saved.
    """
    try:
        with open(input_cdb_path, 'r') as f_in:
            lines = f_in.readlines()
    except FileNotFoundError:
        print(f"Error: The input file was not found at '{input_cdb_path}'")
        return

    new_lines = []
    is_in_etblock = False

    # 1. Process the file line by line
    for line in lines:
        # 2. Detect the start of the ETBLOCK
        #    The .strip() removes leading/trailing whitespace for robust detection.
        if line.strip().startswith('ETBLOCK'):
            is_in_etblock = True
            # 3. Add the replacement line ONCE and continue to the next line
            new_lines.append('ET,1,SHELL181\n')
            continue

        # 4. If we are inside the block, we need to find its end
        if is_in_etblock:
            # The block ends with a line containing only '-1'
            if line.strip() == '-1':
                is_in_etblock = False  # The block has ended, resume normal processing
            # Skip the current line because it's part of the block we are removing
            continue

        # 5. If not inside the ETBLOCK, just add the original line to our new file
        new_lines.append(line)

    # 6. Write the modified content to the new output file
    try:
        with open(output_cdb_path, 'w') as f_out:
            f_out.writelines(new_lines)
        print(f"Successfully processed file. Modified version saved to:\n{output_cdb_path}")
    except IOError as e:
        print(f"Error: Could not write to the output file '{output_cdb_path}'.\nDetails: {e}")


# --- HOW TO USE ---

# 1. Define the name of your original and new CDB files
original_file = 'Design1_mesh.cdb'
modified_file = 'Design1_mesh_MODIFIED.cdb'

# 2. Run the function
replace_etblock_in_cdb(original_file, modified_file)

# Now, use 'Design1_mesh_MODIFIED.cdb' in your PyACP import script.


Successfully processed file. Modified version saved to:
Design1_mesh_MODIFIED.cdb


In [33]:
import pathlib
import numpy as np
import ansys.acp.core as pyacp
from ansys.acp.core import FabricWithAngle, PlyType, SymmetryType, RosetteType
from ansys.acp.core.material_property_sets import (
    ConstantEngineeringConstants, ConstantStrainLimits, ConstantStressLimits
)
#from ansys.acp.core.plotting import get_directions_plotter


In [34]:
# Launch the ACP server
acp = pyacp.launch_acp()

# Define the path to your CDB file
# IMPORTANT: Make sure this path is correct!
cdb_file_path = pathlib.Path(r"C:\Pranav2.0\7\Design1_mesh_MODIFIED.cdb")

# Import the model from the CDB file
model = acp.import_model(
    path=str(cdb_file_path),
    format="ansys:cdb",
    unit_system=pyacp.UnitSystemType.SI  # Using SI units (Pascals, meters, etc.)
)
print("Model imported successfully.")



Model imported successfully.


In [35]:
import pyvista as pv
mesh = model.mesh.to_pyvista()
plotter = pv.Plotter()
plotter.add_mesh(mesh, show_edges=True)
plotter.show()

Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137affeb9b0_10&reconnect=auto" class="pyvi…

In [36]:
# ==============================================================================
# Engineering constants in SI units (Pascals for moduli)
engineering_constants_ud = ConstantEngineeringConstants.from_orthotropic_constants(
    E1=5e10, E2=1e10, E3=1e10,
    nu12=0.28, nu13=0.28, nu23=0.3,
    G12=5e9, G23=4e9, G31=4e9
)

# Strain limits (dimensionless)
strain_limit_tension = 0.01
strain_limit_compression = -0.008
strain_limit_shear = 0.012
strain_limits = ConstantStrainLimits.from_orthotropic_constants(
    eXc=strain_limit_compression, eYc=strain_limit_compression, eZc=strain_limit_compression,
    eXt=strain_limit_tension, eYt=strain_limit_tension, eZt=strain_limit_tension,
    eSxy=strain_limit_shear, eSyz=strain_limit_shear, eSxz=strain_limit_shear
)

# Stress limits in SI units (Pascals)
stress_limits = ConstantStressLimits.from_orthotropic_constants(
    Xt=7.8e8, Yt=3.1e7, Zt=3.1e7,
    Xc=-4.8e8, Yc=-1e8, Zc=-1e8,
    Sxy=3.5e7, Syz=2.5e7, Sxz=3.5e7
)



In [37]:
ud_material = model.create_material(
    name="E-Glass UD",
    ply_type=PlyType.REGULAR,
    engineering_constants=engineering_constants_ud,
    strain_limits=strain_limits,
    stress_limits=stress_limits,
)
print(f"Material '{ud_material.name}' created.")

Material 'E-Glass UD' created.


In [38]:
ud_fabric_0125mm = model.create_fabric(
    name="E-Glass UD 0.125mm",
    material=ud_material,
    thickness=0.000125  # meters
)
print(f"Fabric '{ud_fabric_0125mm.name}' created.")


Fabric 'E-Glass UD 0.125mm' created.


In [39]:
biax_glass_ud = model.create_stackup(
    name="Biax E-Glass UD [0,45,-45,90]s",
    fabrics=[
        FabricWithAngle(ud_fabric_0125mm, 0),
        FabricWithAngle(ud_fabric_0125mm, 45),
        FabricWithAngle(ud_fabric_0125mm, -45),
        FabricWithAngle(ud_fabric_0125mm, 90),
    ],
    symmetry=SymmetryType.EVEN_SYMMETRY,
)
print(f"Stackup '{biax_glass_ud.name}' created.")

Stackup 'Biax E-Glass UD [0,45,-45,90]s' created.


In [40]:
parallel_rosette_45_deg = model.create_rosette(
    name="ParallelRosette",
    rosette_type=RosetteType.PARALLEL,
    origin=[0., 0., 0.],
    # Directions defining a plane rotated 45 degrees around the Y-axis
    dir1=[1, 0., 0],
    dir2=[0, 0., 1],
)
print(f"Rosette '{parallel_rosette_45_deg.name}' created.")



Rosette 'ParallelRosette' created.


In [41]:
oss = model.create_oriented_selection_set(
    name="oss",
    orientation_point=(0.0, 0.0, 0.0),
    orientation_direction=(0.0, 1.0, 0),
    element_sets=[model.element_sets["All_Elements"]],
    rosettes=[parallel_rosette_45_deg],
)

model.update()

In [42]:
from ansys.acp.core import get_directions_plotter
plotter = get_directions_plotter(
    model=model, components=[oss.elemental_data.orientation, oss.elemental_data.reference_direction]
)
plotter.show()

Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137afe11280_11&reconnect=auto" class="pyvi…

In [43]:
mapdl = launch_mapdl(run_location=script_dir, override=True)

In [44]:
# Assume 'model' is your fully defined PyACP model object.
# First, you MUST update the model to apply the layup to the mesh.
model.update()

# Define the path for the output file
# It's best practice to use os.getcwd() to ensure it's in your script's directory
output_dir = script_dir
analysis_model_path = os.path.join(output_dir, "Design1_ACP_Analysis.cdb")

# Export the entire analysis model to a single CDB file.
# This is the key command for the direct transfer workflow [1, 3, 5].
model.export_analysis_model(analysis_model_path)

print(f"Complete analysis model has been exported to:\n{analysis_model_path}")


Complete analysis model has been exported to:
C:\Pranav2.0\8\Design1_ACP_Analysis.cdb


In [45]:
analysis_model_path = os.path.join(script_dir, "Design1_ACP_Analysis.cdb")
# Load the entire composite model using mapdl.input() [1, 5]
mapdl.input(analysis_model_path)
print("Composite model successfully imported into PyMAPDL.")

Composite model successfully imported into PyMAPDL.


In [46]:
# Enter the pre-processor to define loads and boundary conditions
mapdl.prep7()
print("Entered PREP7 processor.")

# --- Get node coordinate information for selection ---
nodes = mapdl.mesh.nodes

Entered PREP7 processor.


In [47]:
mapdl.et('2','MASS21')
mapdl.r('2',1e-12)

REAL CONSTANT SET          2  ITEMS   1 TO   6
   0.10000E-11   0.0000       0.0000       0.0000       0.0000       0.0000

In [48]:
TOL = 1.0E-6
mapdl.allsel('all')
mapdl.get('x_min_model', 'NODE', '0', 'MNLOC','X')
mapdl.get('x_max_model', 'NODE', '0', 'MXLOC','X')
mapdl.get('y_min_model', 'NODE', '0', 'MNLOC','Y')
mapdl.get('y_max_model', 'NODE', '0', 'MXLOC','Y')
mapdl.get('z_level','NODE','0', 'MNLOC','Z')
mapdl.nsel('S','LOC, Z', '(z_level - TOL)', '(z_level + TOL)')


array([3299, 3378, 3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386, 3387,
       3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395, 3396, 3397, 3398,
       3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3409,
       3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420,
       3421, 3422, 3423, 3424, 3425], dtype=int32)

In [49]:
mapdl.nsel('A', 'LOC, X', '(x_min_model - TOL)', '(x_min_model + TOL)')
mapdl.nsel('A', 'LOC, X', '(x_max_model - TOL)', '(x_max_model + TOL)')
mapdl.nsel('A', 'LOC, Y', '(y_min_model - TOL)', '(y_min_model + TOL)')
mapdl.nsel('A', 'LOC, Y', '(y_max_model - TOL)', '(y_max_model + TOL)')

array([3299, 3300, 3301, 3302, 3303, 3304, 3305, 3306, 3307, 3308, 3309,
       3310, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 3319, 3320,
       3321, 3322, 3323, 3324, 3325, 3326, 3327, 3328, 3329, 3330, 3331,
       3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342,
       3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351, 3352, 3353,
       3354, 3355, 3356, 3357, 3358, 3359, 3360, 3361, 3362, 3363, 3364,
       3365, 3366, 3367, 3368, 3369, 3370, 3371, 3372, 3373, 3374, 3375,
       3376, 3377, 3378, 3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386,
       3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395, 3396, 3397,
       3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408,
       3409, 3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419,
       3420, 3421, 3422, 3423, 3424, 3425, 3426, 3427, 3428, 3429, 3430,
       3431, 3432, 3433, 3434, 3435, 3436, 3437, 3438, 3439, 3440, 3441,
       3442, 3443, 3444, 3445, 3446, 3447, 3448, 34

In [50]:
mapdl.nplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137b3562660_12&reconnect=auto" class="pyvi…

In [51]:
mapdl.d('all','all','0')

SPECIFIED CONSTRAINT UX   FOR SELECTED NODES            1 TO        6559 BY           1
 REAL=  0.00000000       IMAG=  0.00000000    
 ADDITIONAL DOFS=  UY    UZ    ROTX  ROTY  ROTZ

In [52]:
mapdl.nplot()
mapdl.nsel('INVE')

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137b3562d80_13&reconnect=auto" class="pyvi…

array([   1,    2,    3, ..., 6557, 6558, 6559],
      shape=(6307,), dtype=int32)

In [53]:
mapdl.nsel('r','ext')

array([   2,   73,  102,  103,  104,  105,  106,  107,  108,  109,  110,
        111,  112,  113,  114,  115,  116,  117,  118,  119,  120,  121,
        122,  123,  124,  125,  126,  127,  128,  129,  130,  131,  132,
        133,  134, 1171, 1235, 1271, 1272, 1273, 1274, 1275, 1276, 1277,
       1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288,
       1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299,
       1300, 1301, 1302, 1303, 2373, 2374, 2375, 2376, 2377, 2378, 2379,
       2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2388, 2389, 2390,
       2391, 2392, 2393, 2394, 2395, 2396, 2397, 2398, 2399, 2400, 2401,
       2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 3042, 3043, 3044,
       3045, 3046, 3047, 3048, 3049], dtype=int32)

In [54]:
M = mapdl.mesh.nodes
X_coord = np.array(M.T[0])
Y_coord = np.array(M.T[1])
Z_coord = np.array(M.T[2])

X_mean = np.mean(X_coord)
Y_mean = np.mean(Y_coord)
Z_mean = np.mean(Z_coord)

mapdl.n(1000000,X_mean,Y_mean,Z_mean)

mapdl.nplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137b9369cd0_14&reconnect=auto" class="pyvi…

In [55]:
mapdl.type('2')
mapdl.real('2')
mapdl.e('1000000')

In [56]:
#I created the remote point but have to connect with the other elements to get the connection
mapdl.rbe3(1000000,'all','all')
mapdl.eplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137b936a9c0_15&reconnect=auto" class="pyvi…

In [57]:
mapdl.allsel()
mapdl.eplot()

[82, 87, 110]


Widget(value='<iframe src="http://localhost:58669/index.html?ui=P_0x137c81d2570_16&reconnect=auto" class="pyvi…

In [73]:
import re
import csv

def save_csv(j, stresses):
    """
    Saves stress data (from a list of lines or a string) to a CSV file for load case j.

    Parameters:
    - j: Load case index (used in file name).
    - stresses: String or list of lines containing the stress data.
    """
    # If stresses is a string, split it into lines
    if isinstance(stresses, str):
        lines = stresses.splitlines()
    else:
        lines = stresses

    # Find the start of data table
    data_start = 0
    for i, line in enumerate(lines):
        if line.strip().startswith('NODE'):
            data_start = i + 1
            break

    # Parse numerical data
    parsed_data = []
    for line in lines[data_start:]:
        line = line.strip()
        if not line or line.startswith('*****'):
            continue

        # Extract numerical values using regex
        values = re.findall(r'[-]?\d*\.\d+E[+-]?\d+|[-]?\d*\.\d+|\d+', line)
        if len(values) >= 7:  # Ensure we have node + 6 stress components
            try:
                node = int(values[0])
                stresses = [float(v) for v in values[1:7]]
                parsed_data.append([node] + stresses)
            except ValueError:
                continue

    # Save to CSV (correct file name: stresses1.csv, stresses2.csv, etc.)
    with open(f'stresses{j+1}.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Node', 'SX', 'SY', 'SZ', 'SXY', 'SYZ', 'SXZ'])
        writer.writerows(parsed_data)

    print(f"CSV file created: stresses{j+1}.csv")


In [76]:
import pandas as pd
forces = pd.read_csv('forces.csv')
l = len(forces)

In [75]:

output_append = []
for i in range(len(forces)):
    mapdl.finish()
    mapdl.slashsolu()
    mapdl.antype("STATIC")
    mapdl.fdele("ALL", "ALL")
    mapdl.f(1000000, 'MZ', forces.iloc[i, 0])
    mapdl.f(1000000, 'MY', forces.iloc[i, 1])
    mapdl.f(1000000, 'MX', forces.iloc[i, 2])
    mapdl.f(1000000, 'FX', forces.iloc[i, 3])
    mapdl.f(1000000, 'FY', forces.iloc[i, 4])
    mapdl.f(1000000, 'FZ', forces.iloc[i, 5])
    output = mapdl.solve()
    output_append.append(output)
    mapdl.post1()
    mapdl.set("LAST")
    stresses = mapdl.prnsol('s','comp')
    save_csv(i,stresses)


CSV file created: stresses1.csv
CSV file created: stresses2.csv
CSV file created: stresses3.csv
CSV file created: stresses4.csv
CSV file created: stresses5.csv
CSV file created: stresses6.csv


In [77]:
mapdl.finish()

EXIT THE MAPDL POST1 DATABASE PROCESSOR


 ***** ROUTINE COMPLETED *****  CP =         0.000

In [78]:
mapdl.exit()

In [94]:
stresses = []

for i in range(l):
    df = pd.read_csv(f'stresses{i+1}.csv')  # assuming files are stresses1.csv, stresses2.csv, ...
    stresses.append(df)

n = len(stresses[0])  # number of rows

Sx = np.zeros(n)
Sy = np.zeros(n)
Sz = np.zeros(n)
Sxy = np.zeros(n)
Syz = np.zeros(n)
Sxz = np.zeros(n)

for i in range(len(stresses)):
    Sx += stresses[i].iloc[:, 1].to_numpy()
    Sy += stresses[i].iloc[:, 2].to_numpy()
    Sz += stresses[i].iloc[:, 3].to_numpy()
    Sxy += stresses[i].iloc[:, 4].to_numpy()
    Syz += stresses[i].iloc[:, 5].to_numpy()
    Sxz += stresses[i].iloc[:, 6].to_numpy()

array([ 4023380., 36501550.,  2191796., ..., -5888750., 11562510.,
       28575150.], shape=(6559,))

In [96]:
nodes = stresses[0].iloc[:, 0]

# Create the new combined DataFrame
combined_df = pd.DataFrame({
    'Node': nodes,
    'Sx': Sx,
    'Sy': Sy,
    'Sz': Sz,
    'Sxy': Sxy,
    'Syz': Syz,
    'Sxz': Sxz
})
combined_df.to_csv('SuperPosition_of_stresses.csv', index=False)
print("CSV file saved as SuperPosition_of_stresses.csv")

CSV file saved as SuperPosition_of_stresses.csv


In [98]:
# Load the components_nodes.csv file
components_nodes = pd.read_csv('components_nodes.csv')

# Load the SuperPosition_of_stresses.csv file
superpositioned_stresses = pd.read_csv('SuperPosition_of_stresses.csv')

# Filter the superpositioned stresses for nodes listed in components_nodes
filtered_stresses = superpositioned_stresses[superpositioned_stresses['Node'].isin(components_nodes['Node'])]

# Merge the filtered stresses with components_nodes to get the component info
merged_data = pd.merge(components_nodes, filtered_stresses, left_on='Node', right_on='Node')

# Select relevant columns: Component, Node, and directional stresses
result = merged_data[['Component', 'Node', 'Sx', 'Sy', 'Sz', 'Sxy', 'Syz', 'Sxz']]

# Write the result to a new CSV file
result.to_csv('Final_superpositioned_stresses.csv', index=False)
